In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf   
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img
from tensorflow.keras.applications import ResNet50,Xception
from tensorflow.keras.models import Sequential,Model   
from tensorflow.keras.layers import Dense,Input, GlobalAveragePooling2D, Dense, Dropout, multiply, Reshape, Conv2D, Activation, Add, Dropout, GlobalAveragePooling2D, BatchNormalization, Multiply,MaxPooling2D,Concatenate,Conv2D, UpSampling2D, Cropping2D,Lambda
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler,ModelCheckpoint

import shutil
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy

In [ ]:
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg

In [ ]:
# Set the image size and batch size for training
batch_size =16

In [ ]:
# Define input shape 
input_shape = (400,400,3)
    

In [ ]:
train_dataset_dir='/kaggle/input/isic-2017-preprocessed-augmented/content/Linear_Exact_Aug/Train'   

In [ ]:
Valid_dataset_dir='/kaggle/input/isic-2017-preprocessed-augmented/content/Linear_Exact_Aug/Valid'   

In [ ]:
# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  
)

In [ ]:
# Load the training dataset with data augmentation
train_generator = train_datagen.flow_from_directory(
    train_dataset_dir,   
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
)

In [ ]:
test_dataset_dir = '/kaggle/input/isic-2017-preprocessed-augmented/content/Linear_Exact_Aug/Test' 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_data_generator = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values to the range [0, 1]
    # Add any other preprocessing options if needed
)

test_generator = test_data_generator.flow_from_directory(
    test_dataset_dir,    
    target_size=(400,400),  # Adjust to match your model's input size
    batch_size=16,           # Adjust batch size as needed   
    class_mode='categorical',  # If you have class labels
    shuffle=False              # Do not shuffle test data
)

In [ ]:
# Data normalization for validation and testing
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)   

In [ ]:
# Load the validation dataset
val_generator = val_datagen.flow_from_directory(
    Valid_dataset_dir,   
    target_size=input_shape[:2],
    batch_size=batch_size,    
    class_mode='categorical'
)

In [ ]:
# Create an input layer
input_layer = Input(shape=input_shape)   

In [ ]:
import tensorflow_hub as hub    

In [ ]:
bit_l_url = "https://tfhub.dev/google/bit/m-r101x1/1"   

In [ ]:
# Load the BiT L model from TensorFlow Hub
bit_l_model = hub.KerasLayer(bit_l_url, trainable=True)
bit_l_output = bit_l_model(input_layer)    

In [ ]:
bit_l_output_reshaped = Reshape((1, 1, -1))(bit_l_output)  # Reshape to match the expected shape

In [ ]:
# Load the Xception model without top classification layers
xception_base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in xception_base_model.layers:
    layer.trainable = True     

In [ ]:
# Connect the input layer to the base models
xception_features = xception_base_model(input_layer)


In [ ]:
combined_features =multiply([xception_features,bit_l_output_reshaped])   

In [ ]:
# Apply Squeeze-and-Excitation block
def se_block(input_tensor):   
    num_channels = input_tensor.shape[-1]
    
    # Squeeze operation (Global Average Pooling)
    squeeze = GlobalAveragePooling2D()(input_tensor)
    squeeze = Reshape((1, 1, num_channels))(squeeze)
    
    # Excitation operation (Fully connected layers)
    excitation = Dense(num_channels // 16, activation='relu')(squeeze)
    excitation = Dense(num_channels, activation='sigmoid')(excitation)
    
    # Scale the input feature maps
    scaled_features = multiply([input_tensor, excitation])
    
    return scaled_features

In [ ]:
se_output = se_block(combined_features)
   

In [ ]:
# Add a classification head to the combined features
from tensorflow.keras.regularizers import l2
x = tf.keras.layers.GlobalAveragePooling2D()(se_output)
x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.1)(x)
x = tf.keras.layers.Dense(2048, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='selu',kernel_regularizer=l2(0.02))(x)
x = BatchNormalization()(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)  # 7output classes


In [ ]:
# Create the ensemble model
model = Model(inputs=input_layer, outputs=output)    

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

# Define your model here
model = model

# Specify the file path where you want to save the model architecture image
image_path = 'model_architecture.png'

# Plot the model architecture and save it as an image
plot_model(model, to_file=image_path, show_shapes=True, show_layer_names=True)

In [ ]:
model.summary()   

In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import categorical_crossentropy

def combined_loss(y_true, y_pred, alpha=0.2, categorical_weight=0.5):
    # Compute the categorical cross-entropy loss
    cat_loss = categorical_crossentropy(y_true, y_pred)

    # Reshape the inputs to get anchor, positive, and negative examples
    anchor = tf.reshape(y_pred[:, 0], shape=(-1, 1))
    positive = tf.reshape(y_pred[:, 1], shape=(-1, 1))
    negative = tf.reshape(y_pred[:, 2], shape=(-1, 1))

    # Compute the distance between the anchor and the positive
    pos_distance = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    
    # Compute the distance between the anchor and the negative
    neg_distance = tf.reduce_sum(tf.square(anchor - negative), axis=1)

    # Compute the triplet loss
    triplet_basic_loss = pos_distance - neg_distance + alpha
    triplet_loss = tf.reduce_mean(tf.maximum(triplet_basic_loss, 0.0), axis=0)

    # Compute the combined loss
    combined_loss = categorical_weight * cat_loss + (1 - categorical_weight) * triplet_loss

    return combined_loss

# Example usage in a Keras model
model.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])


In [ ]:
# Implement learning rate scheduling   
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1)

In [ ]:

# Define the model checkpoint callback to save the best weights
model_checkpoint = ModelCheckpoint('ISIC2017_Classification_contour_ensemble5.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True, verbose=1)

In [ ]:
# Implement early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=True)

In [ ]:
# Train the model
epochs =30
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=epochs,   
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    callbacks=[lr_scheduler,model_checkpoint,early_stopping]
)

In [ ]:
# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')   
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

#Load the saved weights from the output directory in the executed model architecture .  
model.load_weights('/kaggle/working/ISIC2017_Classification_contour_ensemble5.h5')



In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")    
print(f"Test Accuracy: {test_accuracy}")
    

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#  the true labels and predicted labels for the test dataset
y_true = test_generator.classes
y_pred = model.predict(test_generator).argmax(axis=1)
   
# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)
    
# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap='viridis')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
labels = {0:'melanoma', 1:'nevus', 2:'seborrheic_keratosis'}
    

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report
cm = confusion_matrix(y_true, y_pred)
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]   
fig, ax = plt.subplots(figsize=(8,6))  
sns.heatmap(cmn, annot=True, xticklabels=labels.values(), yticklabels=labels.values(),cmap=plt.cm.Blues, fmt='.2f')
plt.ylabel('Actual Classes')
plt.xlabel('Predicted Classes')
plt.show(block=False)    
   
# Generate the classification report
report = classification_report(y_true, y_pred)

print("Confusion Matrix:")
print(cm)    
print(report)     

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
print("Precision: "+ str(precision_score(y_true, y_pred, average='weighted')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='weighted')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
f1 = f1_score(y_true, y_pred, average='weighted')
print("F1 Score: " + str(f1))
     

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred, average='macro')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='macro')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
f1 = f1_score(y_true, y_pred, average='macro')
print("F1 Score: " + str(f1))
    

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred, average='micro')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='micro')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
f1 = f1_score(y_true, y_pred, average='micro')
print("F1 Score: " + str(f1))
         

In [ ]:
from sklearn.metrics import roc_curve, auc
y_true = test_generator.classes   
# Get the probabilities for each class (0 and 1) from the model predictions
y_prob = model.predict(test_generator)

# Compute the ROC curve for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(train_generator.num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true == i, y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
   
# Plot the ROC curves
plt.figure()
for i in range(train_generator.num_classes):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score    
print("weighted Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='weighted')))
print("macro Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='macro')))    
print("micro Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='micro')))  